In [1]:
from pyspark.sql import functions as F
# Read Silver tables
customers_silver = spark.read.table("Silver_Customers")
products_silver  = spark.read.table("Silver_Products")
orders_silver    = spark.read.table("Silver_Orders")

display(customers_silver)
display(products_silver)
display(orders_silver)

StatementMeta(, b634abce-221f-42ff-8450-091829d205c9, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5e202001-1b33-430f-9ab1-7b4e92d786b8)

SynapseWidget(Synapse.DataFrame, abb49059-1d01-438e-9fb2-5dcce4b5f046)

SynapseWidget(Synapse.DataFrame, 8afef9da-1c7a-44d0-b227-d4a7e96affd8)

### **Customer Sales Aggregation**

In [2]:
# Aggregate total revenue, total items, total orders per customer
customer_sales = (orders_silver
    .groupBy("CustomerID", "Name", "Country", "Segment")
    .agg(
        F.countDistinct("OrderID").alias("TotalOrders"),
        F.sum("Quantity").alias("TotalItems"),
        F.sum("Total").alias("TotalRevenue")
    )
    .withColumn("LoadDate", F.current_timestamp())
)

display(customer_sales)

StatementMeta(, b634abce-221f-42ff-8450-091829d205c9, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2b5bb5e1-07e7-45c3-9414-cb43126e4eab)

### **Product Sales Aggregation**

In [3]:
# Aggregate total quantity sold and total revenue per product
product_sales = (orders_silver
    .groupBy("ProductID", "ProductName", "Category")
    .agg(
        F.sum("Quantity").alias("TotalQuantitySold"),
        F.sum("Total").alias("TotalRevenue")
    )
    .withColumn("LoadDate", F.current_timestamp())
)

display(product_sales)


StatementMeta(, b634abce-221f-42ff-8450-091829d205c9, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 8160b8e6-8249-44ef-8574-3981ae5405af)

### **Segment Sales Aggregation**

In [4]:
# Aggregate total revenue by customer segment
segment_sales = (orders_silver
    .groupBy("Segment")
    .agg(
        F.countDistinct("CustomerID").alias("TotalCustomers"),
        F.countDistinct("OrderID").alias("TotalOrders"),
        F.sum("Quantity").alias("TotalItems"),
        F.sum("Total").alias("TotalRevenue")
    )
    .withColumn("LoadDate", F.current_timestamp())
)

display(segment_sales)


StatementMeta(, b634abce-221f-42ff-8450-091829d205c9, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 60820396-a625-44ef-b256-c5276464ea14)

### **Category Sales Aggregation**

In [5]:
# Aggregate total revenue by product category
category_sales = (orders_silver
    .groupBy("Category")
    .agg(
        F.countDistinct("ProductID").alias("TotalProducts"),
        F.sum("Quantity").alias("TotalQuantitySold"),
        F.sum("Total").alias("TotalRevenue")
    )
    .withColumn("LoadDate", F.current_timestamp())
)

display(category_sales)


StatementMeta(, b634abce-221f-42ff-8450-091829d205c9, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 832abc4b-2663-4139-baae-a44c3b8b206a)

In [6]:
# Save Gold tables as Delta for reporting / dashboards
customer_sales.write.mode("overwrite").format("delta").saveAsTable("Gold_CustomerSales")
product_sales.write.mode("overwrite").format("delta").saveAsTable("Gold_ProductSales")
segment_sales.write.mode("overwrite").format("delta").saveAsTable("Gold_SegmentSales")
category_sales.write.mode("overwrite").format("delta").saveAsTable("Gold_CategorySales")

print("✅ Gold layer tables created: Gold_CustomerSales, Gold_ProductSales, Gold_SegmentSales, Gold_CategorySales")


StatementMeta(, b634abce-221f-42ff-8450-091829d205c9, 8, Finished, Available, Finished)

✅ Gold layer tables created: Gold_CustomerSales, Gold_ProductSales, Gold_SegmentSales, Gold_CategorySales
